In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os


# import tensorflow as tf
# import tensorflow.compat.v1 as v1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

# import tensorflow_datasets as tfds

AttributeError: type object 'h5py.h5.H5PYConfig' has no attribute '__reduce_cython__'

In [32]:
import os.path

def build_path(path):
    for i in path:
        if not os.path.exists(i):
            os.makedirs(i)
    

def Normalize(X, axis, savefile= None):
    Xmean, Xstd = X.mean(axis=axis), X.std(axis=axis)
    for i in range(Xstd.size):
        if (Xstd[i]==0):
            Xstd[i]=1
    X = (X - Xmean) / Xstd
    if savefile != None:
        Xmean.tofile(savefile+'mean.bin')
        Xstd.tofile(savefile+'std.bin')
    return X

In [33]:
data = pd.read_csv('output.txt', header=None)

data[0]

0         0.248028
1         0.247512
2         0.247585
3         0.247253
4         0.247683
            ...   
246201   -0.119152
246202   -0.114068
246203   -0.108449
246204   -0.100855
246205   -0.094479
Name: 0, Length: 246206, dtype: float64

In [34]:
X = np.loadtxt('algebra_input.txt', delimiter=',')
Y = np.loadtxt('algebra_output.txt', delimiter=',')

In [35]:
Xmean, Xstd = X.mean(axis=0), X.std(axis=0)
Ymean, Ystd = Y.mean(axis=0), Y.std(axis=0)

for i in range(Xstd.size):
    if (Xstd[i]==0):
        Xstd[i]=1
for i in range(Ystd.size):
    if (Ystd[i]==0):
        Ystd[i]=1
        
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

In [36]:
Xdim  = X.shape[1]
Ydim = Y.shape[1]
samples = X.shape[0]
# print(X.shape, Y.shape)

In [37]:
X_nn = tf.placeholder(tf.float32, [None, Xdim], name='x-input')
Y_nn = tf.placeholder(tf.float32, [None, Ydim], name='y-input')

In [38]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

In [71]:
keep_prob = tf.placeholder(tf.float32)

H0 = X_nn[:,:-1] 
# H0 = tf.expand_dims(H0, -1)       
H0 = tf.nn.dropout(H0, keep_prob=keep_prob)

b0 = bias_variable([512])
W0 = weight_variable([Xdim-1,512])

# print(b0.shape)
# print(W0.shape)
# print(H0.shape)

H1 = tf.matmul(H0, W0) + b0
# print(H1.shape, "lol")
H1 = tf.nn.elu(H1)
H1 = tf.nn.dropout(H1, keep_prob=keep_prob)

layer = tf.keras.layers.LayerNormalization(axis=1, epsilon=0.0000001)
H1 = layer(H1)

b1 = bias_variable([512])
W1 = weight_variable([512,512])

# print(b1.shape)
# print(W1.shape)
# print(H1.shape)

H2 = tf.matmul(H1, W1) + b1
H2 = tf.nn.elu(H2)
H2 = tf.nn.dropout(H1, keep_prob=keep_prob)

layer = tf.keras.layers.LayerNormalization(axis=1, epsilon=0.0000001)
H2 = layer(H2)

b2 = bias_variable([Ydim])
W2 = weight_variable([512,Ydim])

H3 = tf.matmul(H2, W2) + b2
# H3 = tf.squeeze(H3, -1)

print(Y_nn.shape)
print(H3.shape)


cross_entropy = tf.reduce_mean(tf.square(Y_nn - H3), reduction_indices=[1])

loss_shit = tf.reduce_mean(tf.square(Y_nn - H3))

def loss():
    return tf.reduce_mean(tf.square(Y_nn - H3))


(?, 186)
(?, 186)


In [61]:
learning_rate      = 0.001
weightDecay        = 0.001

clr = learning_rate
wdc = weightDecay
# , wdc = [0.0005,0.01]


batch_size         = 32
training_epochs    = 150
Te                 = 16
Tmult              = 2
total_batch        = int(samples / batch_size)

In [62]:
optimizer = tf.compat.v1.train.AdamOptimizer(
    learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False,
    name='Adam'
).minimize(cross_entropy)

In [63]:
# train_step = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.025, beta_2=0.25).minimize(loss, var_list=[b0,W0,b1,W1,b2,W2])

lr_c = tf.placeholder(tf.float32)
wd_c = tf.placeholder(tf.float32)

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.0025).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(H3, 1), tf.argmax(Y_nn, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [64]:
# create a saver
saver = tf.train.Saver()

# initialize the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [65]:
print("total_batch:", total_batch)

total_batch: 403


In [66]:
I = np.arange(samples)

count_test     = .2
num_testBatch  = np.int(total_batch * count_test)
num_trainBatch = total_batch - num_testBatch
print("training_batch:", num_trainBatch)
print("test_batch:", num_testBatch)

training_batch: 323
test_batch: 80


In [67]:
error_train = np.ones((training_epochs,batch_size))
error_test  = np.ones((training_epochs,batch_size))

In [68]:
import numpy as np
# import tensorflow as tf

# tf.set_random_seed(23456)  # reproducibility

class AdamWParameter:
    def __init__(self, nEpochs = 150, Te=10, Tmult=2, LR=0.0001, weightDecay=0.0025, batchSize=32, nBatches = 0):
        self.nEpochs     = nEpochs                #number of total epoch
        self.Te          = Te                     #Ti: total number of epochs within the i-th run / restart of the algorithm
        self.Tmult       = Tmult                  
        self.LR          = LR                     #learning rate
        self.weightDecay = weightDecay            #learning rate decay rate 
        self.batchSize   = batchSize              #bt: batch size                   
        self.nBatches    = nBatches               #number of total batch
        self.EpochNext   = self.Te + 1            #next restart epoch
        self.T_cur       = 0   
        self.t           = 0 
        self.wd          = self.weightDecayNormalized()
        self.H_cur       = 0.9
    
    
    #yita
    def learningRateCosineSGDR(self, epoch):
        self.T_cur = self.T_cur + 1.0 / (self.Te * self.nBatches)
        if self.T_cur >= self.H_cur:
            self.T_cur = self.H_cur
        if (self.T_cur >= self.H_cur) and (epoch == self.EpochNext):
            self.T_cur = 0
            self.Te = self.Te * self.Tmult
            self.EpochNext = self.EpochNext + self.Te
        
        return 0.5 * (1 + np.cos(np.pi * self.T_cur))

    #wt
    def weightDecayNormalized(self):
        return self.weightDecay / (np.power(self.nBatches * self.Te, 0.5))
    
    
    #update and get paramter every epoch
    def getParameter(self, epoch):
        
        yita = self.learningRateCosineSGDR(epoch)
        lr   = yita * self.LR
        clr  = lr/(1+self.t*0)                               #currentLearningRate
        wdc  = yita * self.wd                                #weightDecayCurrent
        self.t +=1 
        return (
                np.float32(clr),
                np.float32(wdc)
                )

In [69]:
ap = AdamWParameter(nEpochs      = training_epochs, 
                    Te           = Te,
                    Tmult        = Tmult,
                    LR           = learning_rate, 
                    weightDecay  = weightDecay,
                    batchSize    = batch_size,
                    nBatches     = total_batch
                    )

In [70]:
print('Learning start..')

TESTNUMBER = 2


for epoch in range(training_epochs):
    avg_cost_train = 0
    avg_cost_test  = 0
    
    for i in range(num_trainBatch):
        index_train = I[i*batch_size:(i+1)*batch_size]
        batch_xs = X[index_train]
        batch_ys = Y[index_train]
#         clr, wdc = ap.getParameter(epoch)   #currentLearningRate & weightDecayCurrent
#         clr,wdc = ap.getParameter(epoch)
    
        feed_dict = {X_nn: batch_xs, Y_nn: batch_ys, keep_prob: 0.7, lr_c: clr, wd_c: wdc}
        l, _, = sess.run([cross_entropy, optimizer], feed_dict=feed_dict)
        avg_cost_train += l / num_trainBatch

    
    for i in range(num_testBatch):
        if i==0:
            index_test = I[-(i+1)*batch_size: ]
        else:
            index_test = I[-(i+1)*batch_size: -i*batch_size]
        batch_xs = X[index_test]
        batch_ys = Y[index_test]
        feed_dict = {X_nn: batch_xs, Y_nn: batch_ys, keep_prob: 1}
        testError = sess.run(cross_entropy, feed_dict=feed_dict)
        avg_cost_test += testError / num_testBatch       
#         if i % 1000 == 0:
#             print(i, "testloss:",testError)
            
            
    print('Epoch:',  '%04d' % (epoch + 1), 'trainingloss =', np.mean(avg_cost_train))
    print('Epoch:', '%04d' % (epoch + 1), 'testloss =', np.mean(avg_cost_test))
    
    with open("training/epochrates"+str(TESTNUMBER)+".txt", "a") as f:
        f.write('Epoch: ' +  '%04d' % (epoch + 1) + ' trainingloss = '+ str(np.mean(avg_cost_train))+"\n")
        f.write('Epoch: ' +  '%04d' % (epoch + 1) + ' testloss = '+ str(np.mean(avg_cost_test))+"\n\n")
    
    error_train[epoch] = avg_cost_train
    error_test[epoch]  = avg_cost_test
    
    error_train.tofile("training/model/error_train.bin")
    error_test.tofile("training/model/error_test.bin")

    save_path = saver.save(sess, "training/model/model.ckpt")
    print("model saved in:", "training/model/model.ckpt")
                     
print('Learning Finished!')

Learning start..
Epoch: 0001 trainingloss = 0.38812393
Epoch: 0001 testloss = 0.56527454
model saved in: training/model/model.ckpt
Epoch: 0002 trainingloss = 0.25476813
Epoch: 0002 testloss = 0.48326188
model saved in: training/model/model.ckpt
Epoch: 0003 trainingloss = 0.22246125
Epoch: 0003 testloss = 0.4471696
model saved in: training/model/model.ckpt
Epoch: 0004 trainingloss = 0.20571963
Epoch: 0004 testloss = 0.4260388
model saved in: training/model/model.ckpt
Epoch: 0005 trainingloss = 0.19571194
Epoch: 0005 testloss = 0.41165453
model saved in: training/model/model.ckpt
Epoch: 0006 trainingloss = 0.18836373
Epoch: 0006 testloss = 0.40037352
model saved in: training/model/model.ckpt
Epoch: 0007 trainingloss = 0.18227336
Epoch: 0007 testloss = 0.39163586
model saved in: training/model/model.ckpt
Epoch: 0008 trainingloss = 0.17831558
Epoch: 0008 testloss = 0.38561475
model saved in: training/model/model.ckpt
Epoch: 0009 trainingloss = 0.17457388
Epoch: 0009 testloss = 0.37964934
m

Epoch: 0073 trainingloss = 0.14132488
Epoch: 0073 testloss = 0.3264734
model saved in: training/model/model.ckpt
Epoch: 0074 trainingloss = 0.14100136
Epoch: 0074 testloss = 0.3260042
model saved in: training/model/model.ckpt
Epoch: 0075 trainingloss = 0.1413863
Epoch: 0075 testloss = 0.32591027
model saved in: training/model/model.ckpt
Epoch: 0076 trainingloss = 0.14117041
Epoch: 0076 testloss = 0.32594112
model saved in: training/model/model.ckpt
Epoch: 0077 trainingloss = 0.14088191
Epoch: 0077 testloss = 0.32652605
model saved in: training/model/model.ckpt
Epoch: 0078 trainingloss = 0.1404904
Epoch: 0078 testloss = 0.32579553
model saved in: training/model/model.ckpt
Epoch: 0079 trainingloss = 0.14106312
Epoch: 0079 testloss = 0.32607615
model saved in: training/model/model.ckpt
Epoch: 0080 trainingloss = 0.14065796
Epoch: 0080 testloss = 0.32594365
model saved in: training/model/model.ckpt
Epoch: 0081 trainingloss = 0.14029187
Epoch: 0081 testloss = 0.32604712
model saved in: trai

model saved in: training/model/model.ckpt
Epoch: 0146 trainingloss = 0.13626654
Epoch: 0146 testloss = 0.32706943
model saved in: training/model/model.ckpt
Epoch: 0147 trainingloss = 0.1372379
Epoch: 0147 testloss = 0.32729584
model saved in: training/model/model.ckpt
Epoch: 0148 trainingloss = 0.1356182
Epoch: 0148 testloss = 0.3272813
model saved in: training/model/model.ckpt
Epoch: 0149 trainingloss = 0.13680899
Epoch: 0149 testloss = 0.3271486
model saved in: training/model/model.ckpt
Epoch: 0150 trainingloss = 0.13647923
Epoch: 0150 testloss = 0.3267081
model saved in: training/model/model.ckpt
Learning Finished!


In [45]:
# 'Epoch:',  '%04d' % (epoch + 1), 'trainingloss =', np.mean(avg_cost_train)

('Epoch:', '0150', 'trainingloss =', 0.13136181)

In [21]:
build_path(['data'])
build_path(['training'])
build_path(['training/nn'])
build_path(['training/weights'])
build_path(['training/model'])
build_path(['results'])

In [163]:
testing = X[:658]

In [164]:
feed_dict = {X_nn : testing, keep_prob : 1}
lol = sess.run(H3, feed_dict=feed_dict)

In [165]:
print(lol.shape)

(658, 186)


In [166]:
lol_01_01 = np.around(lol * Ystd + Ymean, 6)

In [152]:
Y_01_01 = np.around(Y[:658] * Ystd + Ymean, 6)

In [141]:
Y_01_01 = np.loadtxt('output.txt', delimiter=',')[:658]
output_01_01 = np.loadtxt("output_Algebra_01_01.txt", delimiter=",")[30:]

In [153]:
print(output_01_01.shape)
print(Y_01_01.shape)

(658, 186)
(658, 186)


In [170]:
output_01_01 == lol_01_01

print(output_01_01[0,:6], lol_01_01[0,:6])

[ 0.248028  0.52702  -0.568856  0.046869 -0.021253 -0.038151] [ 0.196861  0.527795 -0.409708  0.031153 -0.082547 -0.03182 ]


In [192]:
print(Y_01_01[0,:6])
print(output_01_01[0,:6])

# for i in range(Y_01_01.shape[0]):
#     if sum(Y[i] == output_01_01[50]) > 46:
#         print(sum(Y_01_01[i] == output_01_01[30]))

[ 0.248028  0.52702  -0.568856  0.046869 -0.021253 -0.038151]
[ 0.248028  0.52702  -0.568856  0.046869 -0.021253 -0.038151]


In [193]:
gen_cur = np.array([X[0]])
shit = np.zeros(output_01_01.shape)

for i in range(output_01_01.shape[0]):
    
    gen_cur = np.array([X[i]])
    
    if i > 0:
        gen_cur[0,:186] = gen_nex
    
    feed_dict = {X_nn : gen_cur, keep_prob : 1}
    
    gen_nex = sess.run(H3, feed_dict=feed_dict)
    shit[i] = gen_nex


shit_01_01 = np.around(shit * Ystd + Ymean, 6)

In [200]:
print(sum(sum(abs(output_01_01 - lol_01_01))))
print(sum(sum(abs(output_01_01 - shit_01_01))))



1930.1921010000008
4807.504580999998


In [171]:
with open("results/test.txt", "w+") as f:
    for row in lol_01_01:
        f.write(",".join(format(x, "1.6f") for x in row) + "\n")

In [197]:
with open("results/gen_test.txt", "w+") as f:
    for row in shit_01_01:
        f.write(",".join(format(x, "1.6f") for x in row) + "\n")

In [ ]:
for filename in os.listdir():
    if "output_Algebra_01" in filename:
        print(filename)
        data = pd.read_csv(filename, header = None)
        output = data[30:]